# Plot channel spectra from the HDF store

Plots channel spectra data from provided CSV files

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime
from IPython.display import display, Math, Latex
import sys
import gc
import tables
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)


### Filter out outliers by flux for each device

In [ ]:
# Requires fstore and store open
def filter_by_flux( fkey ):
    print('')
    print('Filtering higher end outliers '+repr(fkey)+' at '+repr(sigmas)+' sigma level')
    flux  = store[fkey].flux
    fstd  = flux.std()
    fmean = flux.mean()

    print(repr(sigmas)+' Sigma range for mean value '+repr(fmean)+' is '+repr(fmean-sigmas*fstd)+' to '+repr(fmean+sigmas*fstd))

    fstore.put(fkey,store[fkey][flux-fmean <= (sigmas*fstd)],format='table',complevel=9, complib='blosc')
    print(repr(len(store[fkey].index)-len(fstore[fkey].index))+' remove and ' + repr(len(fstore[fkey].index))+ ' records left')
    gc.collect()

In [ ]:
sigmas=2
store = pd.HDFStore('moussala.h5',mode='r')
fstore = pd.HDFStore('moussala-filtered.h5',mode='w')
filter_by_flux('liulin')
filter_by_flux('A1')
filter_by_flux('A2')
fstore.close()


# Plot flux spectra after filtering

In [ ]:
try:
    fstore.close()
    store.close()
except:
    pass

fstore = pd.HDFStore('moussala-filtered.h5',mode='r')
store = pd.HDFStore('moussala.h5',mode='r')

In [ ]:
layout = dict(
    title = "Flux histogram",
    xaxis=dict(
        title='Flux [s<sup>-1</sup>cm<sup>-2</sup>]',
    ),
    yaxis=dict(
        title='Probability',
    ),
    barmode='overlay'
)

data_liulin = go.Histogram(x=fstore['liulin'].flux, histnorm='probability',name='Liulin')
data_A1 = go.Histogram(x=fstore['A1'].flux, histnorm='probability',name='A1')
data_A2 = go.Histogram(x=fstore['A2'].flux, histnorm='probability',name='A2')

fig = dict(data=[data_A1,data_A2],layout=layout)

plotly.offline.iplot(fig)

# Compute channel spectrum for Liulin

In [ ]:
# Import Liulin measurements processed by the timestamp format transformation sed script
surface = 2 # cm2
df = pd.read_csv('liulin-all.dat', sep=',', header=0, parse_dates=[0])
df['flux'] = df.iloc[:,range(3,260)].sum(axis=1)/(surface*df.exposition)
df.set_index('timestamp',inplace=True)

# Detect events, where ch1 > 0, e.g. we assume that there was some interference
# Move these to another dataframe
dfn = df[df['ch1'] > 0]

# Remove that noise from the original dataframe
df.drop(df[df['ch1'] > 0].index,inplace=True)

print(np.shape(df))

In [ ]:
## Compute channel histogram for Liulin data
### Data columns are 5..260
liulin_channelHisto=df.iloc[:,3:260].sum(axis=0).T
print(np.shape(liulin_channelHisto))

# Flux plot from data

In [ ]:
def parse_data( fna, chan ):
    surface = 2.0 # cm2
    dtime=10.24 # s
    dfa = pd.read_csv(fna, sep=',', header=None, comment='*', parse_dates=[0], error_bad_lines=False)
    dfa['flux'] = dfa.iloc[:,chan:515].sum(axis=1).astype('float64')/(surface*dtime)
    return dfa


In [ ]:
A1=parse_data('A1-2017-2018-04.csv',264)
A2=parse_data('A2-2017-2018-04.csv',266)

In [ ]:
print('A1: ' + repr(A1.flux.quantile(0.995)))
print('A2: ' + repr(A2.flux.quantile(0.995)))

In [ ]:
layout = dict(
    title = "Flux histogram",
    xaxis=dict(
        title='Flux [s<sup>-1</sup>cm<sup>-2</sup>]',
    ),
    yaxis=dict(
        title='Probability',
    ),
)

#data_liulin = go.Histogram(x=fstore['liulin'].flux, histnorm='probability',name='Liulin')
data_liulin = go.Histogram(x=df.flux[df.flux < df.flux.quantile(0.995)], histnorm='probability',name='Liulin')
data_A1 = go.Histogram(x=A1.flux[A1.flux < A1.flux.quantile(0.995)], histnorm='probability',name='A1')
data_A2 = go.Histogram(x=A2.flux[A2.flux < A2.flux.quantile(0.995)], histnorm='probability',name='A2')

fig = dict(data=[data_A1,data_A2,data_liulin],layout=layout)

plotly.offline.iplot(fig)

# Channel histogram plot

In [ ]:
def compute_channels( fna ):
    #fna = 'A2-test.csv'
    # Number of lines in the file
    fna_chunksize = 100000
    dfa_chunks = pd.read_csv(fna, sep=',', header=None, comment='*', parse_dates=[0], error_bad_lines=False,chunksize=fna_chunksize)

    skip_chunks = 0
    chunk_number = 0
    lines_parsed = 0
    for chunk in dfa_chunks:
        chunk_number = chunk_number + 1
        if (chunk_number > skip_chunks):
            # Drop where flux above norm
            chunk['flux']=chunk.iloc[:,267:515].sum(axis=1).astype('float64')
            chunk.drop(chunk[chunk['flux'] > 0.17*20.48].index, inplace=True)

            chunk_sum=chunk.iloc[:,3:515].sum(axis=0).T
            if (chunk_number==skip_chunks+1):
                dfa_channelHisto=chunk_sum
            else:
                dfa_channelHisto=dfa_channelHisto+chunk_sum
            lines_parsed+=len(chunk.index)
            print(repr(fna)+' parsed '+repr(lines_parsed)+' lines.')
            del chunk_sum
            del chunk
            gc.collect()
            #print('Histo:')
            #print(dfa_channelHisto)
            #print(chunk_number*fna_chunksize,dfa_channelHisto.sum())
            if ((chunk_number-skip_chunks)*fna_chunksize > 1e8):
                break

            # Beware - resetting column index
            #dfa_channelHisto = dfa_channelHisto.reset_index(drop=True)
        else:
            print('Skipping chunk '+repr(chunk_number))
    del dfa_chunks
    print(repr(fna)+' Done '+repr(lines_parsed)+' lines.')
    return dfa_channelHisto

In [ ]:
# Chunked processor
dfa1_channelHisto=compute_channels('A1-2017-2018-04.csv')
dfa2_channelHisto=compute_channels('A2-2017-2018-04.csv')

In [ ]:
layout = dict(
    title = "Channel histogram",
    xaxis=dict(
        #range=[dfa1_channelHisto[dfa_channelHisto != 0.0].first_valid_index(),514],
        range=[0.3,4],
        title='Channel energy [MeV]'
    ),
    yaxis=dict(
        #type='log',
        #autorange=True
        range=[0,10000]
    )
)

liulin_chanwidth=0.0814
liulin_skip=0
data_liulin = go.Scattergl(x=np.arange(0,(256-liulin_skip)*liulin_chanwidth,liulin_chanwidth), y=liulin_channelHisto[liulin_skip:], name='Liulin')

# Bacha, Dasa namita peak
a1min=255
a1emax=6.5
#a1emax=6.0/(2.28/1.47)
print('Estimated energy range: '+repr(a1emax)+' MeV')
data_a1chh = go.Scattergl(x=np.arange(0,a1emax,a1emax/(515-a1min)),y=dfa1_channelHisto[a1min:515], name='A1')

a2min=257
a2emax=a1emax
data_a2chh = go.Scattergl(x=np.arange(0,a2emax,a2emax/(515-a2min)),y=dfa2_channelHisto[a2min:515],name='A2')

fig = dict(data=[data_liulin,data_a1chh,data_a2chh], layout=layout)

plotly.offline.iplot(fig)

In [ ]:
# Subtract noise from first half from second half
a1min=255
data=dfa1_channelHisto[a1min:]
data.reset_index(inplace=True,drop=True)

noise=np.flip(dfa1_channelHisto[:a1min+1],axis=0)
noise.reset_index(inplace=True,drop=True)
data=data-noise
a1data=data[data>0]

# Subtract noise from first half from second half
a2min=257
data=dfa2_channelHisto[a2min:]
data.reset_index(inplace=True,drop=True)

noise=np.flip(dfa2_channelHisto[:a2min+1],axis=0)
noise.reset_index(inplace=True,drop=True)
data=data-noise
a2data=data[data>0]

layout = dict(
    title = "Channel histogram",
    xaxis=dict(
        #range=[dfa1_channelHisto[dfa_channelHisto != 0.0].first_valid_index(),514],
        range=[0.3,4],
        title='Channel energy [MeV]'
    ),
    yaxis=dict(
        #type='log',
        #autorange=True,
        range=[0,20000]
    )
)

liulin_chanwidth=0.0814
liulin_skip=7
data_liulin = go.Scattergl(x=np.arange(0,(256-liulin_skip)*liulin_chanwidth,liulin_chanwidth), y=liulin_channelHisto[7:], name='Liulin')

a1min=255
a1emax=6.0/(2.28/1.47)
print('Estimated energy range: '+repr(a1emax)+'MeV')
data_a1chh = go.Scattergl(x=np.arange(0,a1emax,a1emax/256),y=a1data,name='A1')

a2min=257
a2emax=a1emax
data_a2chh = go.Scattergl(x=np.arange(0,a2emax,a2emax/256),y=a2data, name='A2')

fig = dict(data=[data_liulin,data_a1chh,data_a2chh], layout=layout)

plotly.offline.iplot(fig)

In [ ]:
layout = dict(
    title = "Channel histogram",
    xaxis=dict(
        #range=[dfa1_channelHisto[dfa_channelHisto != 0.0].first_valid_index(),514],
        #range=[230,515],
        title='Channel numbers'
    ),
    yaxis=dict(
        type='log',
        autorange=True
        #range=[0,5000]
    )
)

data_a1chh = go.Scattergl(y=dfa1_channelHisto[0:515], name='A1',mode='markers')
data_a2chh = go.Scattergl(y=dfa2_channelHisto[0:515], name='A2',mode='markers')

fig = dict(data=[data_a1chh,data_a2chh], layout=layout)

plotly.offline.iplot(fig)